<a href="https://colab.research.google.com/github/PetrGavrilin/VKR_URFU/blob/main/VKR_part2_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Подготовка данных</h1>

In [1]:
try:
  import tsfel
except:
  !pip install tsfel >/dev/null 2>&1
  import tsfel

In [2]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter

In [3]:
import sys
sys.path.append('/content')

from vkr_function import my_features_extraction

In [4]:
#Y = pd.read_csv('/content/flow_rate.csv')
sig_up = pd.read_csv('/content/sig_up.csv')
sig_dw = pd.read_csv('/content/sig_dw.csv')
#borders = pd.read_csv('/content/borders.csv').values.tolist()[0]

In [5]:
nX_up = my_features_extraction(sig_up)

*** Feature extraction started ***


/content/vkr_function.py:79: UserWarning: Using default sampling frequency set in configuration file.
  X = tsfel.time_series_features_extractor(cfg_file, s)



*** Feature extraction finished ***


In [6]:
nX_dw = my_features_extraction(sig_dw)

*** Feature extraction started ***


/content/vkr_function.py:79: UserWarning: Using default sampling frequency set in configuration file.
  X = tsfel.time_series_features_extractor(cfg_file, s)



*** Feature extraction finished ***


In [7]:
print(len(nX_up[0]))
print(len(nX_dw[0]))

305
311


In [9]:
X = np.hstack([nX_up, nX_dw])
#Y = np.array(Y['Et'])

In [10]:
#X.tofile('X_tsfel.csv',sep=',')

X_tsfel = pd.DataFrame(np.array(X))
X_tsfel.to_csv('/content/X_tsfel.csv', index=False)

Обучение модели на дополнительно обработанных данных с помощью функции Савицкого-Голея

In [11]:
sig_up_sg = sig_up.apply(lambda x: savgol_filter(x,30,4), axis=0) # сглаженные сигналы 1-го пьезоэлектрического датчика
sig_dw_sg = sig_dw.apply(lambda x: savgol_filter(x,30,4), axis=0) # сглаженные сигналы 2-го пьезоэлектрического датчика

In [12]:
nX_up = my_features_extraction(sig_up_sg)

*** Feature extraction started ***


/content/vkr_function.py:79: UserWarning: Using default sampling frequency set in configuration file.
  X = tsfel.time_series_features_extractor(cfg_file, s)



*** Feature extraction finished ***


In [13]:
nX_dw = my_features_extraction(sig_dw_sg)

*** Feature extraction started ***


/content/vkr_function.py:79: UserWarning: Using default sampling frequency set in configuration file.
  X = tsfel.time_series_features_extractor(cfg_file, s)



*** Feature extraction finished ***


In [14]:
print(len(nX_up[0]))
print(len(nX_dw[0]))

235
271


Осталось гораздо меньше признаков, чем без применения сглаживания

In [15]:
X = np.hstack([nX_up, nX_dw])

In [16]:
#X.tofile('X_tsfel_savgol.csv',sep=',')

X_tsfel_savgol = pd.DataFrame(X)
X_tsfel_savgol.to_csv('/content/X_tsfel_savgol.csv', index=False)

Данные, предварительно обработанные по скользящему среднему

In [26]:
sig_up_rol = sig_up.rolling(30, min_periods=1, axis=0).sum() # скользящее среднее сигналов 1-го пьезоэлектрического датчика
sig_dw_rol = sig_dw.rolling(30, min_periods=1, axis=0).sum() # скользящее среднее сигналов 2-го пьезоэлектрического датчика
sig_up_rol

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,-485.815430,97.485352,485.815430,-103.930664,-7.250977,-154.687500,-274.731445,-144.213867,167.578125,310.986328,...,19.335938,-63.647461,1.611328,-43.505859,-40.283203,-138.574219,-64.453125,-70.092773,71.704102,73.315430
1,-857.226563,219.140625,721.069336,-250.561523,-52.368164,-248.950195,-332.739258,-261.840820,29.809570,219.140625,...,-38.671875,-112.792969,-82.983398,-90.234375,-136.157227,-271.508789,-225.585938,-176.440430,74.926758,124.072266
2,-1463.085938,-74.121094,647.753906,355.297852,506.762695,-849.169922,-775.854492,-597.802734,6.445312,800.830078,...,-166.772461,-232.836914,-203.833008,-188.525391,-309.375000,-468.896484,-430.224609,-295.678711,12.890625,131.323242
3,-2015.771484,-374.633789,533.349609,864.477539,1187.548828,-1529.956055,-1278.588867,-963.574219,-50.756836,1373.657227,...,-288.427734,-374.633789,-376.245117,-285.205078,-489.843750,-655.810547,-665.478516,-435.864258,-40.283203,150.659180
4,-2586.987305,-724.291992,605.053711,1435.693359,1614.550781,-2086.669922,-1715.258789,-1313.232422,-20.947266,1931.982422,...,-328.710938,-528.515625,-480.981445,-381.079102,-593.774414,-865.283203,-865.283203,-571.215820,-25.781250,140.991211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3910,-19242.480469,589.746094,-9821.850586,-4983.837891,1802.270508,13665.673828,-9791.235352,-14051.586914,-12460.400391,-8647.998047,...,-4665.600586,-3918.750000,-5384.252930,-7628.833008,-7524.902344,-5924.047852,-5456.762695,-6203.613281,-7226.000977,-6160.913086
3911,-19196.154785,636.877441,-9886.706543,-4822.302246,1953.332520,13619.348145,-9773.913574,-14093.078613,-12344.787598,-8664.514160,...,-4608.801270,-3879.675293,-5371.765137,-7580.895996,-7446.350098,-5897.863770,-5454.748535,-6158.898926,-7174.841309,-6125.866699
3912,-19348.022461,200.610352,-10407.568359,-5044.262695,2122.119141,14454.418945,-10077.246094,-14472.143555,-12615.893555,-9023.437500,...,-4804.174805,-4100.830078,-5557.470703,-7745.654297,-7628.833008,-6118.212891,-5627.563477,-6312.377930,-7375.048828,-6342.993164
3913,-19109.948730,378.259277,-10064.758301,-4741.735840,2474.597168,13982.702637,-9751.354980,-14282.409668,-12278.723145,-8784.558105,...,-4555.627441,-3890.148926,-5369.348145,-7516.442871,-7371.423340,-5897.058105,-5408.825684,-6130.700684,-7133.752441,-6111.364746


In [27]:
nX_up = my_features_extraction(sig_up_rol)

*** Feature extraction started ***


/content/vkr_function.py:79: UserWarning: Using default sampling frequency set in configuration file.
  X = tsfel.time_series_features_extractor(cfg_file, s)



*** Feature extraction finished ***


In [28]:
nX_dw = my_features_extraction(sig_dw_rol)

*** Feature extraction started ***


/content/vkr_function.py:79: UserWarning: Using default sampling frequency set in configuration file.
  X = tsfel.time_series_features_extractor(cfg_file, s)



*** Feature extraction finished ***


In [29]:
print(len(nX_up[0]))
print(len(nX_dw[0]))

176
214


In [30]:
X = np.hstack([nX_up, nX_dw])

In [31]:
X_tsfel_rol = pd.DataFrame(X)
X_tsfel_rol.to_csv('/content/X_tsfel_rol.csv', index=False)